In [75]:
import numpy as np
import math
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry, CircularAperture, CircularAnnulus 

In [76]:
def calculate_photometry(flux, aperture_area, annulus_area, std, exptime, flux_flag):
    """
    Calculates photometry values including flux error, instrumental magnitude,
    corrected instrumental magnitude, and their errors.
    
    Parameters:
    - flux: float, the measured light intensity of an astronomical object
    - aperture_area: float, the area of the aperture
    - annulus_area: float, the area of the annulus around the aperture
    - std: float, standard deviation (possibly of the background noise)
    - exptime: float, exposure time
    - flux_flag: any type, additional information about the flux measurement
    
    Returns:
    - phot_table: dictionary, containing the calculated photometry values
    """

    # Calculate the flux error
    flux_error = math.sqrt(flux + (aperture_area * (1 + (math.pi * aperture_area) / (2 * annulus_area)) * (std ** 2)))
    
    # Calculate the instrumental magnitude
    inst_mag = -2.5 * math.log10(flux)
    
    # Correct the instrumental magnitude for exposure time
    corr_inst_mag = inst_mag + (2.5 * math.log10(exptime))
    
    # Calculate the error in the instrumental magnitude
    inst_mag_error = (2.5 * flux_error) / (math.log(10) * flux)
    
    # Calculate the logarithm of the instrumental magnitude error
    log_inst_mag_error = math.log10(inst_mag_error)
    
    # Store all the calculated values in a dictionary
    phot_table = {
        "flux": flux,
        "flux_flag": flux_flag,
        "flux_error": flux_error,
        "inst_mag": inst_mag,
        "corr_inst_mag": corr_inst_mag,
        "inst_mag_error": inst_mag_error,
        "log_inst_mag_error": log_inst_mag_error
    }
    
    # Return the dictionary with the photometry values
    return phot_table


In [77]:
# Load the FITS file
file_name = "/Users/wendymendoza/Desktop/05stacked_77.fits"
image_data = fits.getdata(file_name) 

In [78]:
# Define the position of the star (x, y) and radii
x, y = 2043.0361, 2009.0003  # Example coordinates
radius_aperture = 12.45429
radius_inner_annulus = 21.306638
radius_outer_annulus = 30.069002

In [79]:
# Define the apertures
aperture = CircularAperture((x, y), r=radius_aperture)
annulus_aperture = CircularAnnulus((x, y), r_in=radius_inner_annulus, r_out=radius_outer_annulus)
apertures = [aperture, annulus_aperture]

In [80]:
# Get statistics
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)

In [81]:
# Perform aperture photometry
phot_table = aperture_photometry(image_data, apertures)


In [82]:
# Calculate background
bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
bkg_total = bkg_mean * aperture.area

In [83]:
# Subtract background
net_star_flux = phot_table['aperture_sum_0'] - bkg_total

In [84]:
# Calculate magnitude
star_magnitude = -2.5 * np.log10(net_star_flux)

In [85]:
# Call the calculate_photometry function
exptime = 60  # Example exposure time
flux_flag = True  # Example flux flag
photometry_values = calculate_photometry(net_star_flux, aperture.area, annulus_aperture.area, std, exptime, flux_flag)


In [86]:
# Output results
print(f'Star flux: {net_star_flux} e-/s')
print(f'Star magnitude: {star_magnitude}')
print(f'Photometry values: {photometry_values}')



Star flux:   aperture_sum_0 
-----------------
153887.2560243106 e-/s
Star magnitude:    aperture_sum_0  
-------------------
-12.968006639450609
Photometry values: {'flux': <Column name='aperture_sum_0' dtype='float64' length=1>
153887.2560243106, 'flux_flag': True, 'flux_error': 3683.154219485122, 'inst_mag': -12.968006639450609, 'corr_inst_mag': -8.522628513491501, 'inst_mag_error': <Column name='aperture_sum_0' dtype='float64' length=1>
0.02598612768279479, 'log_inst_mag_error': -1.585258431976063}


Spanish version

In [87]:
# Importar librerías
import numpy as np
import math
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from photutils import aperture_photometry, CircularAperture, CircularAnnulus

In [88]:
def calculate_photometry(flux, aperture_area, annulus_area, std, exptime, flux_flag):
    """
    Calculates photometry values including flux error, instrumental magnitude,
    corrected instrumental magnitude, and their errors.
    
    Parameters:
    - flux: float, the measured light intensity of an astronomical object
    - aperture_area: float, the area of the aperture
    - annulus_area: float, the area of the annulus around the aperture
    - std: float, standard deviation (possibly of the background noise)
    - exptime: float, exposure time
    - flux_flag: any type, additional information about the flux measurement
    
    Returns:
    - phot_table: dictionary, containing the calculated photometry values
    """
    # Check for non-positive flux values
    if flux <= 0:
        print("Flux is less than or equal to zero!")
        return

    # Calculate the flux error
    flux_error = math.sqrt(flux + (aperture_area * (1 + (math.pi * aperture_area) / (2 * annulus_area)) * (std ** 2)))
    
    # Calculate the instrumental magnitude
    inst_mag = -2.5 * math.log10(flux)
    
    # Correct the instrumental magnitude for exposure time
    corr_inst_mag = inst_mag + (2.5 * math.log10(exptime))
    
    # Calculate the error in the instrumental magnitude
    inst_mag_error = (2.5 * flux_error) / (math.log(10) * flux)
    
    # Calculate the logarithm of the instrumental magnitude error
    log_inst_mag_error = math.log10(inst_mag_error)
    
    # Store all the calculated values in a dictionary
    phot_table = {
        "flux": flux,
        "flux_flag": flux_flag,
        "flux_error": flux_error,
        "inst_mag": inst_mag,
        "corr_inst_mag": corr_inst_mag,
        "inst_mag_error": inst_mag_error,
        "log_inst_mag_error": log_inst_mag_error
    }
    
    # Return the dictionary with the photometry values
    return phot_table



In [89]:
# Cargar el archivo FITS
file_name = "/Users/wendymendoza/Desktop/05stacked_77.fits"
image_data = fits.getdata(file_name)

In [90]:
# Definir la posición de las estrellas [(x1, y1), (x2, y2), ...] y los radios
star_positions = [(2052.861, 2014.141), (1943.740, 1676.496), (2569.059, 2219.694), (2703.428, 2573.036), (2238.112, 2421.248), (1723.030, 1953.444), (3148.033, 1145.038), (863.159, 2163.909), (3020.855, 452.018), (2090.472, 1772.211), (445.340, 2308.053), (1002.597, 784.886), (2918.320, 3162.513), (1580.905, 3407.705), (3884.231, 2791.008), (265.780, 2174.311), (3903.549, 897.823), (1609.387, 815.844), (3219.238, 945.871), (1567.531, 157.787)]  # Coordenadas de ejemplo para múltiples estrellas
radius_aperture = 12.45429
radius_inner_annulus = 21.306638
radius_outer_annulus = 30.069002

In [91]:
# Obtener estadísticas
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)


In [92]:
# Obtener estadísticas
mean, median, std = sigma_clipped_stats(image_data, sigma=3.0)

# Iterar sobre cada estrella
for position in star_positions:
    x, y = position
    
    # Definir las aperturas
    aperture = CircularAperture((x, y), r=radius_aperture)
    annulus_aperture = CircularAnnulus((x, y), r_in=radius_inner_annulus, r_out=radius_outer_annulus)
    apertures = [aperture, annulus_aperture]
    
    # Realizar fotometría de apertura
    phot_table = aperture_photometry(image_data, apertures)
    
    # Calcular fondo
    bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area
    bkg_total = bkg_mean * aperture.area
    
    # Restar fondo
    net_star_flux = phot_table['aperture_sum_0'] - bkg_total
    
    # Calcular magnitud
    star_magnitude = -2.5 * np.log10(net_star_flux)
    
    # Llamar a la función calculate_photometry
    exptime = 60  # Tiempo de exposición de ejemplo
    flux_flag = True  # Indicador de flujo de ejemplo
    photometry_values = calculate_photometry(net_star_flux, aperture.area, annulus_aperture.area, std, exptime, flux_flag)
    
    # Mostrar resultados
    print(f'Estrella en posición {position}:')
    print(f'Flujo de estrella: {net_star_flux} e-/s')
    print(f'Magnitud de estrella: {star_magnitude}')
    print(f'Valores de fotometría: {photometry_values}\n')

Estrella en posición (2052.861, 2014.141):
Flujo de estrella:   aperture_sum_0 
-----------------
72585.50821092981 e-/s
Magnitud de estrella:    aperture_sum_0  
-------------------
-12.152124804771677
Valores de fotometría: {'flux': <Column name='aperture_sum_0' dtype='float64' length=1>
72585.50821092981, 'flux_flag': True, 'flux_error': 3672.100659935356, 'inst_mag': -12.152124804771677, 'corr_inst_mag': -7.706746678812569, 'inst_mag_error': <Column name='aperture_sum_0' dtype='float64' length=1>
0.05492739160029316, 'log_inst_mag_error': -1.260211024326474}

Estrella en posición (1943.74, 1676.496):
Flujo de estrella:   aperture_sum_0 
-----------------
66587.47431618837 e-/s
Magnitud de estrella:    aperture_sum_0  
-------------------
-12.058481355697165
Valores de fotometría: {'flux': <Column name='aperture_sum_0' dtype='float64' length=1>
66587.47431618837, 'flux_flag': True, 'flux_error': 3671.2838657345656, 'inst_mag': -12.058481355697165, 'corr_inst_mag': -7.613103229738056

/var/folders/k9/yk7m65wn4rn1db4nhhln_w480000gn/T/ipykernel_56620/36143527.py:24: RuntimeWarning: invalid value encountered in log10
  star_magnitude = -2.5 * np.log10(net_star_flux)
/var/folders/k9/yk7m65wn4rn1db4nhhln_w480000gn/T/ipykernel_56620/36143527.py:24: RuntimeWarning: invalid value encountered in log10
  star_magnitude = -2.5 * np.log10(net_star_flux)
/var/folders/k9/yk7m65wn4rn1db4nhhln_w480000gn/T/ipykernel_56620/36143527.py:24: RuntimeWarning: invalid value encountered in log10
  star_magnitude = -2.5 * np.log10(net_star_flux)
/var/folders/k9/yk7m65wn4rn1db4nhhln_w480000gn/T/ipykernel_56620/36143527.py:24: RuntimeWarning: invalid value encountered in log10
  star_magnitude = -2.5 * np.log10(net_star_flux)
/var/folders/k9/yk7m65wn4rn1db4nhhln_w480000gn/T/ipykernel_56620/36143527.py:24: RuntimeWarning: invalid value encountered in log10
  star_magnitude = -2.5 * np.log10(net_star_flux)
/var/folders/k9/yk7m65wn4rn1db4nhhln_w480000gn/T/ipykernel_56620/36143527.py:24: RuntimeWa